## Import libraries and load data

In [455]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import pickle
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [682]:
# Read Data

filepath = '/Users/kevinzhao/Kaggle/Recruit_Restaurant/inputs/'

air_reserve = pd.read_csv(filepath + 'air_reserve.csv')
hpg_reserve = pd.read_csv(filepath + 'hpg_reserve.csv')
air_store_info = pd.read_csv(filepath + 'air_store_info.csv')
hpg_store_info = pd.read_csv(filepath + 'hpg_store_info.csv')
air_visit_data = pd.read_csv(filepath + 'air_visit_data.csv')

date_info = pd.read_csv(filepath + 'date_info.csv')
store_id_relation = pd.read_csv(filepath + 'store_id_relation.csv')
sample_submission = pd.read_csv(filepath + 'sample_submission.csv')

## Merge dataframes

In [683]:
# Join Store reservation and store info dataframes 

air_data = air_reserve.merge(air_store_info, on='air_store_id', how='left')
hpg_data = hpg_reserve.merge(hpg_store_info, on='hpg_store_id', how='left')

In [684]:
# Add hpg ids to air data and air ids to hpg data

air_data = air_data.merge(store_id_relation, on ='air_store_id', how='left')
hpg_data = hpg_data.merge(store_id_relation, on ='hpg_store_id', how='left')

In [685]:
# Merge data sets 

air_data_full = air_data.merge(hpg_data, on =['air_store_id', 'reserve_datetime', 'visit_datetime'], how='left')
data_full = hpg_data.merge(air_data, on = ['hpg_store_id', 'reserve_datetime', 'visit_datetime'], how='outer')

In [460]:
air_data_full.shape, data_full.shape

((92427, 15), (2088053, 15))

Not all air store has a hpg store and vice versa...must combine both dataframes after imputation

In [533]:
print(air_data_full[(air_data_full.hpg_store_id_x.isnull()==True) & (air_data_full.hpg_store_id_y.isnull()==True)].shape)
print(air_data_full[(air_data_full.hpg_store_id_x.isnull()==True) & (air_data_full.hpg_store_id_y.isnull()==False)].shape)
print(air_data_full[(air_data_full.hpg_store_id_x.isnull()==False) & (air_data_full.hpg_store_id_y.isnull()==True)].shape)
print(air_data_full[(air_data_full.hpg_store_id_x.isnull()==False) & (air_data_full.hpg_store_id_y.isnull()==False)].shape)

(56114, 15)
(0, 15)
(31573, 15)
(4740, 15)


In [534]:
air_data_full[(air_data_full.hpg_store_id_x.isnull()==False) & (air_data_full.hpg_store_id_y.isnull()==False)][air_data_full.hpg_store_id_x!=air_data_full.hpg_store_id_y]

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id_x,hpg_store_id_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y


hpg_store_id_y always matches hpg_store_id_x when the value is not NaN

In [686]:
# Rename columns
air_data_full = air_data_full.rename(columns= {'reserve_visitors_x' : 'air_reserve_visitors', 'latitude_x': 'latitude',
                                              'longitude_x':'longitude', 'reserve_visitors_y': 'hpg_reserve_visitors',
                                              'hpg_store_id_x':'hpg_store_id'})

# Drop redundant columns
air_data_full = air_data_full.drop(columns={'hpg_store_id_y', 'latitude_y', 'longitude_y'})

In [536]:
print(air_data_full[(air_data_full.air_area_name.isnull()==True) & (air_data_full.hpg_area_name.isnull()==True)].shape)
print(air_data_full[(air_data_full.air_area_name.isnull()==True) & (air_data_full.hpg_area_name.isnull()==False)].shape)
print(air_data_full[(air_data_full.air_area_name.isnull()==False) & (air_data_full.hpg_area_name.isnull()==True)].shape)
print(air_data_full[(air_data_full.air_area_name.isnull()==False) & (air_data_full.hpg_area_name.isnull()==False)].shape)

(0, 12)
(0, 12)
(90284, 12)
(2143, 12)


`air_area_name` is always present when `hpg_area_name` is present. Most data has `air_area_name` but not `hpg_area_name`.

In [687]:
#Drop hpg_area_name column because no information is added
air_data_full = air_data_full.drop(columns={'hpg_area_name'})

In [538]:
air_data_full.columns

Index(['air_store_id', 'visit_datetime', 'reserve_datetime',
       'air_reserve_visitors', 'air_genre_name', 'air_area_name', 'latitude',
       'longitude', 'hpg_store_id', 'hpg_reserve_visitors', 'hpg_genre_name'],
      dtype='object')

## Impute missing values

Impute missing `hpg_reserve_visitors` values as `air_reserve_visitors`

In [539]:
# air_data_full['hpg_reserve_visitors'] = air_data_full.apply(
#     lambda row: np.int64(row['air_reserve_visitors']) if math.isnan(row['hpg_reserve_visitors'])==True 
#     else np.int64(row['hpg_reserve_visitors']),
#     axis=1)

Impute missing `hpg_reserve_visitors` values as 0

In [688]:
air_data_full['hpg_reserve_visitors'] = air_data_full.apply(
    lambda row: 0 if math.isnan(row['hpg_reserve_visitors'])==True 
    else np.int64(row['hpg_reserve_visitors']),
    axis=1)

Impute missing `hpg_genre_name` as blank

In [689]:
air_data_full['hpg_genre_name'] = air_data_full.apply(
    lambda row: ' ' if type(air_data_full.hpg_genre_name)!=str
    else row['hpg_genre_name'],
    axis=1)

Create dataframe from `sample_submissions` to add onto `air_visit_data`

In [690]:
sample_submission.loc[:,'air_store_id'] = sample_submission.apply(
    lambda row: str(row['id'])[:-11], axis=1)
sample_submission.loc[:,'visit_date'] = sample_submission.apply(
    lambda row: datetime.strptime(row['id'][-10:], '%Y-%m-%d').date(), axis=1)

In [691]:
sample_submission.loc[:,'visitors'] = sample_submission.apply(
    lambda row: np.nan, axis=1)

In [692]:
sample_submission.drop('id', axis=1, inplace=True)

In [693]:
#function to impute missing values from air to hpg
def fill_air_hpg(data, feature, is_string=False):
    """
    Fills missing values from one data set to another.
    :param data: dataframe to impute
    :param feature: feature on the dataframe to impute
    :param is_string: True if feature is string, false if feature is float
    """
    
    feature_x = feature + '_x'
    feature_y = feature + '_y'
    
    #if feature type is string like hpg_store_id, check for NaNs with type
    if is_string == True:
        data[feature_x] = data.apply(
            lambda row: row[feature_y] if type(row[feature_x])!=str else row[feature_x], axis=1)
        data[feature_y] = data.apply(
            lambda row: row[feature_x] if type(row[feature_y])!=str else row[feature_y], axis=1)
    #if feature type is float, check for NaNs using math.isnan()
    else:
        data[feature_x] = data.apply(
            lambda row: row[feature_y] if math.isnan(row[feature_x])==True else row[feature_x], axis=1)
        data[feature_y] = data.apply(
            lambda row: row[feature_x] if math.isnan(row[feature_y])==True else row[feature_y], axis=1)

In [694]:
#function to impute more general missing values from air to hpg
def fill_general(data, feature):
    """
    Fills missing genre and area name from one data set to another.
    :param data: dataframe to impute
    :param feature: feature on the dataframe to impute
    """
    
    air_feature = 'air_'+feature
    hpg_feature = 'hpg_'+feature
    
    data[air_feature] = data.apply(
        lambda row: row[hpg_feature] if type(row[air_feature])!=str else row[air_feature], axis=1)
    data[hpg_feature] = data.apply(
        lambda row: row[air_feature] if type(row[hpg_feature])!=str else row[hpg_feature], axis=1)

## Clean up features

Reformat from string to datetime

In [695]:
from datetime import datetime
from dateutil.parser import parse

In [696]:
air_data_full.loc[:,'visit_datetime'] = air_data_full.apply(
    lambda row: datetime.strptime(row['visit_datetime'], '%Y-%m-%d %H:%M:%S'), axis=1)
air_data_full.loc[:,'reserve_datetime'] = air_data_full.apply(
    lambda row: datetime.strptime(row['reserve_datetime'], '%Y-%m-%d %H:%M:%S'), axis=1)

In [697]:
air_data_full.loc[:,'visit_date'] = air_data_full.apply(
    lambda row: row['visit_datetime'].date(), axis=1)
air_data_full.loc[:,'reserve_date'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].date(), axis=1)

## Aggregate data

Group by restaurant and visit date. Merge with total visits dataframe.

In [698]:
air_data_visits = air_data_full.loc[:,['air_store_id','hpg_store_id','visit_date','reserve_date',
                                      'air_reserve_visitors','hpg_reserve_visitors']].groupby(
                                        ['air_store_id','visit_date'], as_index=False).agg('sum')

In [699]:
df_air_visit_data = air_visit_data.groupby(['air_store_id','visit_date'], as_index=False).agg('max')
df_air_visit_data['visit_date'] = df_air_visit_data.apply(
    lambda row: datetime.strptime(row['visit_date'], '%Y-%m-%d').date(), axis=1)

In [639]:
df_air_visit_data = pd.concat([df_air_visit_data, sample_submission])

In [640]:
df_air = air_data_visits.merge(df_air_visit_data, on=['air_store_id','visit_date'], how='outer')

In [641]:
#rename columns on date_info
date_info = date_info.rename(columns={'calendar_date':'visit_date'})

#convert datetime format
date_info['visit_date'] = date_info.apply(
    lambda row: datetime.strptime(row['visit_date'], '%Y-%m-%d').date(), axis=1)

In [644]:
#merge date information
df_air = df_air.merge(date_info, on='visit_date', how='right')

Add remaining features- genre name, area name, latitude, longitude

In [645]:
air_features = air_data_full.loc[:,['air_store_id', 'air_genre_name', 'hpg_genre_name',
                                    'air_area_name','latitude','longitude']].groupby(
                                    'air_store_id', as_index=False).agg('max')

In [646]:
df_air = df_air.merge(air_features, on='air_store_id', how='left')
df_air.head()

,air_store_id,visit_date,air_reserve_visitors,hpg_reserve_visitors,visitors,day_of_week,holiday_flg,air_genre_name,hpg_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,2016-10-31,2.0,2.0,26.0,Monday,0,Italian/French,,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_03963426c9312048,2016-10-31,12.0,0.0,17.0,Monday,0,Izakaya,,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386245,132.455018
2,air_04cae7c1bc9b2a0b,2016-10-31,13.0,0.0,27.0,Monday,0,Izakaya,,Hokkaidō Sapporo-shi Kotoni 2 Jō,43.074470,141.300889
3,air_0867f7bebad6a649,2016-10-31,2.0,0.0,12.0,Monday,0,Italian/French,,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386245,132.455018
4,air_08cb3c4ee6cd6a22,2016-10-31,20.0,0.0,32.0,Monday,0,Izakaya,,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,34.756950,134.841177


In [647]:
# pickle.dump(df_air, open('df_air.p', 'wb'))
df_air = pickle.load(open('df_air.p', 'rb'))

## Feature Engineering

#### Date and Time

In [490]:
air_data_full.loc[:,'visit_year'] = air_data_full.apply(
    lambda row: row['visit_datetime'].year, axis=1)
air_data_full.loc[:,'visit_month'] = air_data_full.apply(
    lambda row: row['visit_datetime'].month, axis=1)
air_data_full.loc[:,'visit_day'] = air_data_full.apply(
    lambda row: row['visit_datetime'].day, axis=1)
# air_data_full.loc[:,'visit_date'] = air_data_full.apply(
#     lambda row: row['visit_datetime'].date(), axis=1)
air_data_full.loc[:,'visit_hour'] = air_data_full.apply(
    lambda row: row['visit_datetime'].hour, axis=1)

In [491]:
air_data_full.loc[:,'reserve_year'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].year, axis=1)
air_data_full.loc[:,'reserve_month'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].month, axis=1)
air_data_full.loc[:,'reserve_day'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].day, axis=1)
# air_data_full.loc[:,'reserve_date'] = air_data_full.apply(
#     lambda row: row['reserve_datetime'].date(), axis=1)
air_data_full.loc[:,'reserve_hour'] = air_data_full.apply(
    lambda row: row['reserve_datetime'].hour, axis=1)

In [492]:
air_data_full.columns

Index(['air_store_id', 'visit_datetime', 'reserve_datetime',
       'air_reserve_visitors', 'air_genre_name', 'air_area_name', 'latitude',
       'longitude', 'hpg_store_id', 'hpg_reserve_visitors', 'hpg_genre_name',
       'visit_date', 'reserve_date', 'visit_year', 'visit_month', 'visit_day',
       'visit_hour', 'reserve_year', 'reserve_month', 'reserve_day',
       'reserve_hour'],
      dtype='object')

In [493]:
#reorder columns
air_columns = ['air_store_id', 'hpg_store_id','visit_date','reserve_date','visit_datetime','reserve_datetime', 'air_reserve_visitors',
               'hpg_reserve_visitors', 'air_genre_name', 'hpg_genre_name','air_area_name','latitude','longitude',
               'visit_year','visit_month','visit_day','reserve_year','reserve_month','reserve_day','visit_hour',
                 'reserve_hour']
air_data_full = air_data_full[air_columns]

In [494]:
air_data_full.head()

,air_store_id,hpg_store_id,visit_date,reserve_date,visit_datetime,reserve_datetime,air_reserve_visitors,hpg_reserve_visitors,air_genre_name,hpg_genre_name,...,latitude,longitude,visit_year,visit_month,visit_day,reserve_year,reserve_month,reserve_day,visit_hour,reserve_hour
0,air_877f79706adbfb06,NaN,2016-01-01,2016-01-01,2016-01-01 19:00:00,2016-01-01 16:00:00,1,0,Japanese food,,...,35.658068,139.751599,2016,1,1,2016,1,1,19,16
1,air_db4b38ebe7a7ceff,hpg_bcecaa70222d94e5,2016-01-01,2016-01-01,2016-01-01 19:00:00,2016-01-01 19:00:00,3,0,Dining bar,,...,34.676231,135.486059,2016,1,1,2016,1,1,19,19
2,air_db4b38ebe7a7ceff,hpg_bcecaa70222d94e5,2016-01-01,2016-01-01,2016-01-01 19:00:00,2016-01-01 19:00:00,6,0,Dining bar,,...,34.676231,135.486059,2016,1,1,2016,1,1,19,19
3,air_877f79706adbfb06,NaN,2016-01-01,2016-01-01,2016-01-01 20:00:00,2016-01-01 16:00:00,2,0,Japanese food,,...,35.658068,139.751599,2016,1,1,2016,1,1,20,16
4,air_db80363d35f10926,hpg_878cc70b1abc76f7,2016-01-01,2016-01-01,2016-01-01 20:00:00,2016-01-01 01:00:00,5,0,Dining bar,,...,43.770635,142.364819,2016,1,1,2016,1,1,20,1


In [237]:
# pickle dataframe to load faster
# pickle.dump(air_data_full, open('air_data_full.p', 'wb'))
air_data_full = pickle.load(open('air_data_full.p', 'rb'))

#### Genre Name

First combine `air_genre_name` and `hpg_genre_name`

In [648]:
df_air.loc[:, 'genre_name'] = df_air.apply(
    lambda row: (row['air_genre_name'] + ' ' + row['hpg_genre_name']) 
    if (type(row['air_genre_name']) == str) and (type(row['hpg_genre_name']) == str)
    else row['air_genre_name'], axis=1)

In [649]:
#add feature for restaurants with Japanese in the name
df_air.loc[:,'Japanese_explicit'] = df_air.apply(
    lambda row: 1 if 'Japanese' in str(row.genre_name) else 0, axis=1)

In [650]:
#add feature for restaurants with Japanese food
df_air.loc[:,'Japanese'] = df_air.apply(
    lambda row: 1 if ('Japanese' in str(row.genre_name) or 'Izakaya' in str(row.genre_name)
                      or 'Sukiyaki' in str(row.genre_name) or 'Teppanyaki' in str(row.genre_name)
                      or 'Sushi' in str(row.genre_name)) 
                else 0, axis=1)

In [651]:
#add feature for Italian restaurants
df_air.loc[:,'Italian'] = df_air.apply(
    lambda row: 1 if ('Italian' in str(row.genre_name) or 'Pizza' in str(row.genre_name)) else 0, axis=1)

In [652]:
#add feature for French restaurants
df_air.loc[:,'French'] = df_air.apply(
    lambda row: 1 if 'French' in str(row.genre_name) else 0, axis=1)

In [653]:
#add feature for Korean restaurants
df_air.loc[:,'Korean'] = df_air.apply(
    lambda row: 1 if 'Korean' in str(row.genre_name) else 0, axis=1)

In [654]:
#add feature for restaurants with Chinese food
df_air.loc[:,'Chinese'] = df_air.apply(
    lambda row: 1 if ('Chinese' in str(row.genre_name) or 'Cantonese' in str(row.genre_name)
                      or 'Shanghai' in str(row.genre_name) or 'Sichuan' in str(row.genre_name)
                      or 'Dim Sum' in str(row.genre_name) or 'Taiwanese' in str(row.genre_name)) 
                else 0, axis=1)

In [655]:
#add feature for restaurants with Western food
df_air.loc[:,'Western'] = df_air.apply(
    lambda row: 1 if ('Western' in str(row.genre_name) or 'Italian' in str(row.genre_name)
                      or 'French' in str(row.genre_name) or 'Spanish' in str(row.genre_name)
                      or 'Spain' in str(row.genre_name) or 'Taiwanese' in str(row.genre_name)) 
                else 0, axis=1)

In [656]:
#add feature for restaurants with Chinese food
df_air.loc[:,'Asian'] = df_air.apply(
    lambda row: 1 if (row.Japanese == 1 or row.Korean == 1 or row.Chinese == 1) 
                else 0, axis=1)

In [657]:
df_air.genre_name.unique()

array(['Italian/French  ', 'Izakaya  ', 'Western food  ', 'Dining bar  ',
       'Yakiniku/Korean food  ', 'Bar/Cocktail  ', 'Japanese food  ',
       'Cafe/Sweets  ', 'Creative cuisine  ',
       'Okonomiyaki/Monja/Teppanyaki  ', nan, 'Asian  ', 'Other  ',
       'Karaoke/Party  ', 'International cuisine  '], dtype=object)

In [658]:
df_air.head()

,air_store_id,visit_date,air_reserve_visitors,hpg_reserve_visitors,visitors,day_of_week,holiday_flg,air_genre_name,hpg_genre_name,air_area_name,...,longitude,genre_name,Japanese_explicit,Japanese,Italian,French,Korean,Chinese,Western,Asian
0,air_00a91d42b08b08d9,2016-10-31,2.0,2.0,26.0,Monday,0,Italian/French,,Tōkyō-to Chiyoda-ku Kudanminami,...,139.753595,Italian/French,0,0,1,1,0,0,1,0
1,air_03963426c9312048,2016-10-31,12.0,0.0,17.0,Monday,0,Izakaya,,Hiroshima-ken Hiroshima-shi Kokutaijimachi,...,132.455018,Izakaya,0,1,0,0,0,0,0,1
2,air_04cae7c1bc9b2a0b,2016-10-31,13.0,0.0,27.0,Monday,0,Izakaya,,Hokkaidō Sapporo-shi Kotoni 2 Jō,...,141.300889,Izakaya,0,1,0,0,0,0,0,1
3,air_0867f7bebad6a649,2016-10-31,2.0,0.0,12.0,Monday,0,Italian/French,,Hiroshima-ken Hiroshima-shi Kokutaijimachi,...,132.455018,Italian/French,0,0,1,1,0,0,1,0
4,air_08cb3c4ee6cd6a22,2016-10-31,20.0,0.0,32.0,Monday,0,Izakaya,,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,...,134.841177,Izakaya,0,1,0,0,0,0,0,1


In [659]:
pickle.dump(df_air, open('df_air2.p', 'wb'))
df_air = pickle.load(open('df_air2.p', 'rb'))

In [660]:
df_air = df_air.sort_values(['air_store_id', 'visit_date'])

#### Area Name

- Prefecture
- City

#### Dummy Variables

In [664]:
df_air = pd.concat([df_air.drop('day_of_week', axis=1),(pd.get_dummies(df_air.day_of_week))], axis=1)

In [665]:
pickle.dump(df_air, open('df_air2.p','wb'))

In [190]:
air_data_full.head()

,air_store_id,hpg_store_id,visit_datetime,reserve_datetime,air_reserve_visitors,hpg_reserve_visitors,air_genre_name,hpg_genre_name,air_area_name,latitude,...,reserve_hour,genre_name,Japanese_explicit,Italian,Korean,Chinese,French,Western,Japanese,Asian
0,air_877f79706adbfb06,NaN,2016-01-01 19:00:00,2016-01-01 16:00:00,1,1,Japanese food,NaN,Tōkyō-to Minato-ku Shibakōen,35.658068,...,16,Japanese food,1,0,0,0,0,0,1,1
1,air_db4b38ebe7a7ceff,hpg_bcecaa70222d94e5,2016-01-01 19:00:00,2016-01-01 19:00:00,3,3,Dining bar,NaN,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,...,19,Dining bar,0,0,0,0,0,0,0,0
2,air_db4b38ebe7a7ceff,hpg_bcecaa70222d94e5,2016-01-01 19:00:00,2016-01-01 19:00:00,6,6,Dining bar,NaN,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,...,19,Dining bar,0,0,0,0,0,0,0,0
3,air_877f79706adbfb06,NaN,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2,Japanese food,NaN,Tōkyō-to Minato-ku Shibakōen,35.658068,...,16,Japanese food,1,0,0,0,0,0,1,1
4,air_db80363d35f10926,hpg_878cc70b1abc76f7,2016-01-01 20:00:00,2016-01-01 01:00:00,5,5,Dining bar,NaN,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,...,1,Dining bar,0,0,0,0,0,0,0,0


In [681]:
sample_submission

,visitors,air_store_id,visit_date
0,NaN,air_00a91d42b08b08d9,2017-04-23
1,NaN,air_00a91d42b08b08d9,2017-04-24
2,NaN,air_00a91d42b08b08d9,2017-04-25
3,NaN,air_00a91d42b08b08d9,2017-04-26
4,NaN,air_00a91d42b08b08d9,2017-04-27
5,NaN,air_00a91d42b08b08d9,2017-04-28
6,NaN,air_00a91d42b08b08d9,2017-04-29
7,NaN,air_00a91d42b08b08d9,2017-04-30
8,NaN,air_00a91d42b08b08d9,2017-05-01
9,NaN,air_00a91d42b08b08d9,2017-05-02
